In [1]:
import os
import random
from time import time
import pickle
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageOps, ImageEnhance
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, concatenate, Activation, Add, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import gdown
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'

from IPython.display import clear_output
def open_png_or_jpeg(filepath):
  try:
    img = Image.open(filepath + '.png')
  except:
    img = Image.open(filepath + '.jpeg')
  return img


In [2]:
"""""
COLAB
"""""
gdown.download('https://cups.mail.ru/media/tasks/round_622/task_1089/15d655a4e56991fdc78b0923c59c2183', 'sample_submission.csv', True)
gdown.download('https://cups.mail.ru/media/tasks/round_622/task_1089/15d655a4e56991fdc78b0923c59c2183_9lYnJHK', 'train.csv', True)
gdown.download('https://cups.mail.ru/media/tasks/round_622/task_1089/15d655a4e56991fdc78b0923c59c2183_6oHvy5X', 'avia-test.zip', True)
gdown.download('https://cups.mail.ru/media/tasks/round_622/task_1089/15d655a4e56991fdc78b0923c59c2183_iP4xUuJ', 'avia-train.zip', True)
!unzip -q 'avia-test.zip'
!unzip -q 'avia-train.zip'


df_train = pd.read_csv('train.csv')                                 ## train csv path
df_test = pd.read_csv('sample_submission.csv')                      ## test ext csv
avia_train = '/content/avia-train/'                                 ## train folder path
avia_test = '/content/avia-test/'                                   ## test folder path

In [3]:
df_train['fullfilename'] = df_train.filename + '.png'
df_train = df_train.dropna()

contrast = 87  # 87,8,12,59,0,1,6,77,11,96 best 5 by val_auc

METRICS = [
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.AUC(name='auc'),]

In [4]:
x_train = np.array([np.asarray(ImageOps.autocontrast(Image.open(avia_train+i).convert('RGB'), contrast))/255 for i in tqdm(df_train.fullfilename)])
y_train = np.array([i for i in df_train.sign]).reshape((-1,1))

#для аугментатора / база из чистых самолетов
df_train_planes = df_train.loc[df_train.sign == 1]
x_train_planes = np.array([np.asarray(ImageOps.autocontrast(Image.open(avia_train+i).convert('RGB'), contrast))/255 for i in df_train_planes.fullfilename])
y_train_planes = np.array([i for i in df_train_planes.sign]).reshape((-1,1))

x_train.shape, y_train.shape, np.where(y_train==0)[0].shape, np.where(y_train==1)[0].shape 

100%|██████████| 31080/31080 [00:23<00:00, 1317.07it/s]


((31080, 20, 20, 3), (31080, 1), (23181,), (7899,))

# аугментация


In [6]:
for i in range(2):
  generator = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)
  augment_images = generator.flow(x_train_planes, batch_size=7899)
  aug_x = augment_images.next()
  aug_y = np.ones((7899,1))
  x_train = np.append(x_train, aug_x, axis=0)
  y_train = np.append(y_train, aug_y, axis=0)
  aug_x.shape, aug_y.shape, x_train.shape, y_train.shape, np.where(y_train==0)[0].shape, np.where(y_train==1)[0].shape

((7899, 20, 20, 3),
 (7899, 1),
 (46878, 20, 20, 3),
 (46878, 1),
 (23181,),
 (23697,))

# convnet

In [7]:
def create_model():
  x_input = Input(shape=(20,20,3), name='x_input')
  x = Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', name='conv_1_1')(x_input)
  x = Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', name='conv_1_2')(x)
  x = Conv2D(filters=32, kernel_size=5, strides=2, padding='same', activation='relu', name='conv_1_3')(x)
  x = Dropout(0.1)(x)
  x_1 = Flatten(name='flatten_1')(x)

  x = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', name='conv_2_1')(x)
  x = BatchNormalization(name='batch_normalization_2_1')(x)
  x = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', name='conv_2_2')(x)
  x = BatchNormalization(name='batch_normalization_2_2')(x)
  x = Conv2D(filters=64, kernel_size=5, strides=2, padding='same', activation='relu', name='conv_2_3')(x)
  x = BatchNormalization(name='batch_normalization_2_3')(x)
  x = Dropout(0.1)(x)
  x_2 = Flatten(name='flatten_2')(x)

  x = Conv2D(filters=128, kernel_size=3, padding='same', activation='relu', name='conv_3_1')(x)
  x = BatchNormalization(name='batch_normalization_3_1')(x)
  x = Conv2D(filters=128, kernel_size=3, padding='same', activation='relu', name='conv_3_2')(x)
  x = BatchNormalization(name='batch_normalization_3_2')(x)
  x = Conv2D(filters=128, kernel_size=5, strides=2, padding='same', activation='relu', name='conv_3_3')(x)
  x = BatchNormalization(name='batch_normalization_3_3')(x)
  x = Dropout(0.1)(x)
  x_3 = Flatten(name='flatten_3')(x)
  x_conc = concatenate([x_1, x_2, x_3])
  x = Dense(448, activation="relu")(x_conc)
  x = Dense(224, activation="relu")(x)
  x = BatchNormalization(name='batch_normalization_D')(x)
  x_output = Dense(1, activation="sigmoid")(x)
  model = Model(inputs=x_input, outputs=x_output)
  return model

# check contrast effect

In [ ]:
df = pd.DataFrame()
for n in tqdm(range(100)):
  x_train, y_train = np.array([np.asarray(ImageOps.autocontrast(Image.open(avia_train+i).convert('RGB'),  n ))/255 for i in df_train.fullfilename]), np.array([i for i in df_train.sign]).reshape((-1,1))
  model = create_model()
  model.compile(loss="binary_crossentropy", optimizer = Adam(learning_rate=(0.001)), metrics=[METRICS])
  history = model.fit(x_train, y_train, epochs=5, verbose=0, validation_split=0.3)
  tail = pd.DataFrame(history.history).tail(1)
  tail['contrast'] = n
  df = df.append(tail)

In [ ]:
df.sort_values('val_auc').tail(10) 

,loss,fp,fn,auc,val_loss,val_fp,val_fn,val_auc,contrast
4,0.073790,162.0,143.0,0.996009,0.082354,26.0,124.0,0.995477,96
4,0.044066,104.0,85.0,0.998340,0.130016,222.0,6.0,0.995570,11
4,0.068305,149.0,130.0,0.996766,0.081014,107.0,39.0,0.995584,77
4,0.048956,113.0,90.0,0.998276,0.088862,137.0,20.0,0.995662,6
4,0.043598,92.0,72.0,0.998122,0.097909,146.0,20.0,0.995754,1
4,0.051579,116.0,103.0,0.997887,0.066977,53.0,70.0,0.995782,0
4,0.063103,144.0,127.0,0.997210,0.080819,126.0,23.0,0.996231,59
4,0.038670,86.0,74.0,0.998735,0.077280,113.0,25.0,0.996293,12
4,0.046362,92.0,79.0,0.998180,0.070437,80.0,25.0,0.996335,8
4,0.056099,119.0,95.0,0.997385,0.054287,47.0,54.0,0.997365,87


# train

In [ ]:
model = create_model()
model.compile(loss="binary_crossentropy", optimizer = Adam(learning_rate=(0.001)), metrics=[METRICS])
history = model.fit(x_train, y_train, epochs=5, verbose=1)

Epoch 1/5
969/969 [==============================] - 31s 23ms/step - loss: 0.1820 - fp: 1179.0000 - fn: 988.0000 - auc: 0.9793
Epoch 2/5
969/969 [==============================] - 21s 22ms/step - loss: 0.0966 - fp: 598.0000 - fn: 506.0000 - auc: 0.9935
Epoch 3/5
969/969 [==============================] - 21s 22ms/step - loss: 0.0759 - fp: 471.0000 - fn: 403.0000 - auc: 0.9959
Epoch 4/5
969/969 [==============================] - 21s 22ms/step - loss: 0.0585 - fp: 331.0000 - fn: 310.0000 - auc: 0.9972
Epoch 5/5
969/969 [==============================] - 22s 23ms/step - loss: 0.0471 - fp: 267.0000 - fn: 225.0000 - auc: 0.9979


In [ ]:
model.compile(loss="binary_crossentropy", optimizer = Adam(learning_rate=(0.0001)), metrics=[METRICS])
history = model.fit(x_train, y_train, epochs=6, verbose=1)

Epoch 1/6
969/969 [==============================] - 23s 22ms/step - loss: 0.0178 - fp: 362.0000 - fn: 302.0000 - auc: 0.9988
Epoch 2/6
969/969 [==============================] - 22s 22ms/step - loss: 0.0104 - fp: 48.0000 - fn: 45.0000 - auc: 0.9998
Epoch 3/6
969/969 [==============================] - 22s 22ms/step - loss: 0.0074 - fp: 44.0000 - fn: 30.0000 - auc: 0.9998
Epoch 4/6
969/969 [==============================] - 22s 22ms/step - loss: 0.0054 - fp: 25.0000 - fn: 15.0000 - auc: 0.9999
Epoch 5/6
969/969 [==============================] - 22s 22ms/step - loss: 0.0052 - fp: 21.0000 - fn: 26.0000 - auc: 0.9999
Epoch 6/6
969/969 [==============================] - 21s 22ms/step - loss: 0.0029 - fp: 15.0000 - fn: 8.0000 - auc: 0.9999


In [ ]:
imgs = []
for i in range(len(df_test)):
  img = ImageOps.autocontrast(open_png_or_jpeg(avia_test + df_test['filename'][i]).convert('RGB'), contrast)
  img = np.asarray(img)/255
  imgs.append(img)
chosenmodel = model
pred = chosenmodel.predict(np.array(imgs))
df_test['sign'] = pred[:,0]
time0 = round(time(), 0)
df_test.to_csv(f'Google/test_planes{time0}.csv', index=False)
model.save(f'Google/model{time0}.h5')